In [1]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

from datetime import datetime

In [2]:
mlflow_tracking_uri = 'sqlite:///mlflow.db'

client = MlflowClient(tracking_uri=mlflow_tracking_uri)

In [3]:
experiments_list = client.list_experiments()
print(experiments_list)

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>, <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='duration-prediction', tags={}>, <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='test-experiment', tags={}>, <Experiment: artifact_location='./mlruns/3', experiment_id='3', lifecycle_stage='active', name='bike-duration-prediction', tags={}>, <Experiment: artifact_location='./mlruns/4', experiment_id='4', lifecycle_stage='active', name='bike-rental-experiment', tags={}>, <Experiment: artifact_location='file:///C:/Users/HannaPethani/OneDrive%20-%20Honeysuckle%20Health/Documents/GitHub/mlops-project/mlruns', experiment_id='5', lifecycle_stage='active', name='bikeshare-experiment', tags={}>]


In [12]:
experiment = mlflow.get_experiment_by_name('bikeshare-experiment')
print(experiment)
# experiment_id = experiment.experiment_id
# print(experiment_id)

runs = client.search_runs(
    experiment_ids = '5',
    filter_string = 'metrics.rmse < 12.48',
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ['metrics.rmse ASC']
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

Traceback (most recent call last):
  File "c:\Users\HannaPethani\anaconda3\envs\deployment-env\lib\site-packages\mlflow\store\tracking\file_store.py", line 262, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "c:\Users\HannaPethani\anaconda3\envs\deployment-env\lib\site-packages\mlflow\store\tracking\file_store.py", line 341, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\Users\HannaPethani\anaconda3\envs\deployment-env\lib\site-packages\mlflow\utils\file_utils.py", line 181, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file 'C:\Users\HannaPethani\OneDrive - Honeysuckle Health\Documents\GitHub\mlops-project\mlruns\246204cb29094cfc88aca5045bf7d995\meta.yaml' does not exist.
Traceback (most recent call last):
  File "c:\Users\HannaPethani\anaconda3\envs\deployment-env\lib\site-packages\mlflow\store\tracking\fi

None
run id: dcaa866194e64680823a45c547fa47d5, rmse: 12.4639
run id: d9b0ca0ef3df4b67a83673461d01f9ca, rmse: 12.4649
run id: 173e805f721442cda5aac365f14eb848, rmse: 12.4657
run id: c614e50af1834e1181934acac9495f6f, rmse: 12.4661
run id: 4d7d0fc83045441a8fcee4b0de288416, rmse: 12.4666


In [13]:
model_name = 'sklearn-ridge-model'
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f'version: {version.version}, stage: {version.current_stage}')

version: 1, stage: Staging
version: 6, stage: None


In [ ]:
mlflow.set_tracking_uri = (mlflow_tracking_uri)
mlflow.set_experiment('bike-rental-experiment')

run_id = 'ba79aba996a74a48986e51f45d32aea2'
model_uri = f'./mlruns/4/{run_id}/artifacts/models'
mlflow.register_model(model_uri = model_uri, name = 'sklearn-ridge-model')

In [ ]:
model_version = 1
new_stage = 'Production'
client.transition_model_version_stage(
    name = model_name,
    version = model_version,
    stage = new_stage,
    archive_existing_versions = False
)

In [ ]:
date = datetime.today().date()
client.update_model_version(
    name = model_name,
    version = model_version,
    description = f'The model version {model_version} was transitioned to {new_stage} on {date}'
)